In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import time
import sklearn
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import random
from tabulate import tabulate

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier

evaluate={'Acc':"Accuracy", 'b_Acc':"Balanced Accuracy", 'F1':"F1 Score", 'kap':"Kappa", 'ROC':"Roc"}

In [3]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


In [4]:
def folder(f_name): #this function creates a folder named "attacks" in the program directory.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("The folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

files_add=find_the_way("SM",".csv")
files_add

['SM\\DoS-SYN-1.csv',
 'SM\\DoS-SYN-2.csv',
 'SM\\DoS-SYN-3.csv',
 'SM\\DoS-SYN-4.csv',
 'SM\\DoS-SYN-5.csv',
 'SM\\DoS-SYN-6.csv',
 'SM\\MB-ACK-1.csv',
 'SM\\MB-ACK-2.csv',
 'SM\\MB-ACK-3.csv',
 'SM\\MB-ACK-4.csv',
 'SM\\MB-BF-1.csv',
 'SM\\MB-BF-2.csv',
 'SM\\MB-BF-3.csv',
 'SM\\MB-BF-4.csv',
 'SM\\MB-BF-5.csv',
 'SM\\MB-HDis-1.csv',
 'SM\\MB-HDis-2.csv',
 'SM\\MB-HDis-3.csv',
 'SM\\MB-HDis-4.csv',
 'SM\\MB-HDis-5.csv',
 'SM\\MB-HTTP-1.csv',
 'SM\\MB-HTTP-2.csv',
 'SM\\MB-HTTP-3.csv',
 'SM\\MB-HTTP-4.csv',
 'SM\\MB-UDP-1.csv',
 'SM\\MB-UDP-2.csv',
 'SM\\MB-UDP-3.csv',
 'SM\\MB-UDP-4.csv',
 'SM\\MitM-ARP-1.csv',
 'SM\\MitM-ARP-2.csv',
 'SM\\MitM-ARP-3.csv',
 'SM\\MitM-ARP-4.csv',
 'SM\\MitM-ARP-5.csv',
 'SM\\MitM-ARP-6.csv',
 'SM\\Scan-HDis-1.csv',
 'SM\\Scan-HDis-2.csv',
 'SM\\Scan-HDis-3.csv',
 'SM\\Scan-HDis-4.csv',
 'SM\\Scan-HDis-5.csv',
 'SM\\Scan-HDis-6.csv',
 'SM\\Scan-OS-1.csv',
 'SM\\Scan-OS-2.csv',
 'SM\\Scan-OS-3.csv',
 'SM\\Scan-OS-4.csv',
 'SM\\Scan-OS-5.csv',
 'SM\\Scan

In [6]:
ml_list={"DT":DecisionTreeClassifier()}#,"SVC":SVC()}}

In [7]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [8]:
folder("results/compare/SS/")
folder("results/compare/CV/")
folder("results/compare/DD/")

## Calculation of evaluations

In [9]:
def score(train_time,test_time,predict,y_test,class_based_results,repeat,cv,dname,ml,sw):
    train_time=train_time[0]
    test_time=test_time[0]
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,sw,repeat,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return class_based_results,line

In [10]:
def ML(loop1,loop2,output_csv,cols,dname,sw):
    fold=5
    repetition=25
    for ii in ml_list:
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        lines=[["Attack","Feature","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
        for i in range(repetition):

                df = pd.read_csv(loop1,usecols=cols)#,header=None )
                df=df.fillna(0)
                X_train =df[df.columns[0:-1]]
                X_train=np.array(X_train)
                df[df.columns[-1]] = df[df.columns[-1]].astype('category')
                y_train=df[df.columns[-1]].cat.codes  


                df = pd.read_csv(loop2,usecols=cols)#,header=None )
                df=df.fillna(0)
                X_test =df[df.columns[0:-1]]
                X_test=np.array(X_test)
                df[df.columns[-1]] = df[df.columns[-1]].astype('category')
                y_test=df[df.columns[-1]].cat.codes  


                #dname=loop1  [6:-13]  
                results_y=[]
       
                results_y.append(y_test)


                precision=[]
                recall=[]
                f1=[]
                accuracy=[]
                train_time=[]
                test_time=[]
                total_time=[]
                kappa=[]
                accuracy_b=[]

                    #machine learning algorithm is applied in this section
                clf = ml_list[ii]#choose algorithm from ml_list dictionary
                second=time.time()
                clf.fit(X_train, y_train)
                train_time.append(float((time.time()-second)) )
                second=time.time()
                predict =clf.predict(X_test)
                test_time.append(float((time.time()-second)) )
    
                altime=0
                class_based_results,line=score(train_time,test_time,predict,y_test,class_based_results,cv,i,dname,ii,sw)
                lines.append(line)

        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
       
        results=results.mean()
        results=results.round(3)
        #print (tabulate(results, headers=list(results.columns)))
        #print()
        return list(results.values)






In [11]:
def ML_CV(loop1,loop2,output_csv,cols,dname,sw):
    fold=5
    repetition=5
    for ii in ml_list:
        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        lines=[["Attack","Feature","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
        for i in range(repetition):

            rnd = random()
            
            kfold = sklearn.model_selection.KFold(n_splits=fold, shuffle=True, random_state=int(rnd*100))  
            cv=0
            df = pd.read_csv(loop1,usecols=cols)#,header=None )
            ##df = df.reset_index(drop=True)
            df=df.fillna(0)

            #del df["MAC"] # if dataset has MAC colomn please uncomment this line
            X =df[df.columns[0:-1]]
            X=np.array(X)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y=df[df.columns[-1]].cat.codes  
            X.shape
            for train_index, test_index in kfold.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]  

                #dname=loop1  [6:-13]  
                results_y=[]
                cv+=1
                results_y.append(y_test)


                precision=[]
                recall=[]
                f1=[]
                accuracy=[]
                train_time=[]
                test_time=[]
                total_time=[]
                kappa=[]
                accuracy_b=[]

                    #machine learning algorithm is applied in this section
                clf = ml_list[ii]#choose algorithm from ml_list dictionary
                second=time.time()
                clf.fit(X_train, y_train)
                train_time.append(float((time.time()-second)) )
                second=time.time()
                predict =clf.predict(X_test)
                test_time.append(float((time.time()-second)) )
    
                altime=0
                class_based_results,line=score(train_time,test_time,predict,y_test,class_based_results,cv,i,dname,ii,sw)
                lines.append(line)

        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)

        results=results.mean()
        results=results.round(3)
        #print (tabulate(results, headers=list(results.columns)))
        #print()
        return list(results.values)






In [12]:
features=['random', 'ts',#, #'Ether_dst', 'Ether_src', 'IP_src', 'IP_dst', 
        'pck_size', 'Ether_type', 'LLC_dsap', 'LLC_ssap', 'LLC_ctrl', 'EAPOL_version', 'EAPOL_type', 'EAPOL_len', 'IP_version', 'IP_ihl', 'IP_tos', 'IP_len', 'IP_flags', 'IP_Z', 'IP_MF', 'IP_id', 'IP_chksum', 'IP_DF', 'IP_frag', 'IP_ttl', 'IP_proto', 'IP_options', 'ICMP_type', 'ICMP_code', 'ICMP_chksum', 'ICMP_id', 'ICMP_seq', 'ICMP_ts_ori', 'ICMP_ts_rx', 'ICMP_ts_tx', 'ICMP_ptr', 'ICMP_reserved', 'ICMP_length', 'ICMP_nexthopmtu', #'ICMP_unused',
         'TCP_seq', 'TCP_ack', 'TCP_dataofs', 'TCP_reserved', 'TCP_flags', 'TCP_FIN', 'TCP_SYN', 'TCP_RST', 
 'TCP_PSH', 'TCP_ACK', 'TCP_URG', 'TCP_ECE', 'TCP_CWR', 'TCP_window', 'TCP_chksum', 'TCP_urgptr', 'TCP_options', 'UDP_len', 'UDP_chksum', 'DHCP_options', 'BOOTP_op', 'BOOTP_htype', 'BOOTP_hlen', 'BOOTP_hops', 'BOOTP_xid', 'BOOTP_secs', 'BOOTP_flags', 'BOOTP_sname', 'BOOTP_file', 'BOOTP_options', 'DNS_length', 'DNS_id', 'DNS_qr', 'DNS_opcode', 'DNS_aa', 'DNS_tc', 'DNS_rd', 'DNS_ra', 'DNS_z', 'DNS_ad', 'DNS_cd', 'DNS_rcode', 'DNS_qdcount', 'DNS_ancount', 'DNS_nscount', 'DNS_arcount', 'sport_class', 'dport_class', 'sport23', 'dport23', 'sport_bare', 'dport_bare', 'payload_bytes', 'entropy', 'Protocol',
 'dst_IP_diversity', 'dst_port_diversity', 'src_IP_diversity', 'IP_add_count', 'sport', 'dport', #'ID',
         'pck_size_diff', 'pck_size_mean_WE', 'pck_size_std_WE', 'pck_size_sum_of_EW', 'ts_diff', 'ts_mean_WE', 'ts_std_WE', 'ts_sum_of_EW', 'TCP_window_diff', 'TCP_window_mean_WE', 'TCP_window_std_WE', 'TCP_window_sum_of_EW', 'payload_bytes_diff', 'payload_bytes_mean_WE', 'payload_bytes_std_WE', 'payload_bytes_sum_of_EW', 'entropy_diff', 'entropy_mean_WE', 'entropy_std_WE', 'entropy_sum_of_EW', 'pck_size_mean_2', 'pck_size_std_2', 'ts_mean_2', 'ts_std_2', 'TCP_window_mean_2', 'TCP_window_std_2', 'payload_bytes_mean_2',
 'payload_bytes_std_2', 'entropy_mean_2', 'entropy_std_2', 'dport_sum', 'sport_sum', 'TCP_FIN_sum', 'TCP_SYN_sum', 'TCP_RST_sum', 'TCP_PSH_sum', 'TCP_ACK_sum', 'TCP_URG_sum', 'TCP_ECE_sum', 'TCP_CWR_sum', 'TCP_FIN_ratio', 'TCP_SYN_ratio', 'TCP_RST_ratio', 'TCP_PSH_ratio', 'TCP_ACK_ratio', 'TCP_URG_ratio', 'TCP_ECE_ratio', 'TCP_CWR_ratio', 'sum', 'TCP_FIN_SR', 'TCP_SYN_SR', 'TCP_RST_SR', 'TCP_PSH_SR', 'TCP_ACK_SR', 'TCP_URG_SR', 'TCP_ECE_SR', 'TCP_CWR_SR', 'pck_size_mean_6', 'pck_size_std_6', 'ts_mean_6', 'ts_std_6', 'TCP_window_mean_6', 'TCP_window_std_6', 'payload_bytes_mean_6', 'payload_bytes_std_6',
 'entropy_mean_6', 'entropy_std_6', 'pck_size_mean_9', 'pck_size_std_9', 'ts_mean_9', 'ts_std_9', 'TCP_window_mean_9', 'TCP_window_std_9', 'payload_bytes_mean_9', 'payload_bytes_std_9', 'entropy_mean_9', 'entropy_std_9']


In [13]:
file_list={'SM\\DoS-SYN-6.csv':"SYN",
           'SM\\MB-ACK-4.csv':"ACK",
           'SM\\MB-BF-3.csv':"BF",
           'SM\\MB-HTTP-4.csv':"HTTP",
           'SM\\MB-UDP-3.csv':"UDP",
          'SM\\MitM-ARP-4.csv':"ARP",
           'SM\\Scan-HDis-3.csv':"SCHD",
          'SM\\Scan-Port-3.csv':"SP",
          'SM\\Scan-OS-3.csv':"OS",
          'SM\\MB-HDis-3.csv':"MHDis"}



In [16]:
for train in (file_list):
    lines=[["Attack","Feature","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
    for dname in tqdm(features):
        try:
            target_names=["attack","benign"]
            feature=[dname, 'Label']
            output_csv=f"./results/compare/CV/DT_{file_list[train]}_{dname}.csv"
            #print(f"{list[train]} Dataset - Feature {number+1}/{len(features)}")
            temp=ML_CV(train,"",output_csv,feature,dname,file_list[train])
            temp=temp[2:]
            temp=[file_list[train],dname,"DT"]+temp
            lines.append(temp)
            
        except:
            print("#"*110)
            print(f"ERROR ABOUT {list[train]} Dataset - Feature {dname}")
            print("#"*110+"\n\n")

    results = pd.DataFrame (lines[1:], columns = lines[0])
    print (tabulate(results, headers=list(results.columns)))


100%|████████████████████████████████████████████████████████████████████████████████| 174/174 [01:49<00:00,  1.58it/s]


     Attack    Feature                  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T
---  --------  -----------------------  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------
  0  SYN       random                   DT    0.499    0.498   0.498  0.498  0.498  -0.004  0.498    0.063     0
  1  SYN       ts                       DT    0.543    0.548   0.705  0.548  0.434   0.093  0.548    0.003     0
  2  SYN       pck_size                 DT    0.998    0.998   0.998  0.998  0.998   0.996  0.998    0.002     0
  3  SYN       Ether_type               DT    0.524    0.5     0.262  0.5    0.344   0      0.5      0         0
  4  SYN       LLC_dsap                 DT    0.524    0.5     0.262  0.5    0.344   0      0.5      0.001     0
  5  SYN       LLC_ssap                 DT    0.524    0.5     0.262  0.5    0.344   0      0.5      0.001     0
  6  SYN       LLC_ctrl                 DT    0.524    0.5     0.262  0.5    0.344   0  

100%|████████████████████████████████████████████████████████████████████████████████| 174/174 [02:19<00:00,  1.25it/s]


     Attack    Feature                  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T
---  --------  -----------------------  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------
  0  ACK       random                   DT    0.867    0.497   0.497  0.497  0.497  -0.006  0.497    0.048     0
  1  ACK       ts                       DT    0.896    0.568   0.58   0.568  0.573   0.146  0.568    0.01      0
  2  ACK       pck_size                 DT    0.959    0.978   0.817  0.978  0.876   0.754  0.978    0.001     0
  3  ACK       Ether_type               DT    0.929    0.5     0.465  0.5    0.482   0      0.5      0         0
  4  ACK       LLC_dsap                 DT    0.929    0.5     0.465  0.5    0.482   0      0.5      0.001     0
  5  ACK       LLC_ssap                 DT    0.929    0.5     0.465  0.5    0.482   0      0.5      0.002     0
  6  ACK       LLC_ctrl                 DT    0.929    0.5     0.465  0.5    0.482   0  

100%|████████████████████████████████████████████████████████████████████████████████| 174/174 [01:34<00:00,  1.84it/s]


...

     Attack    Feature                  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T
---  --------  -----------------------  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------
  0  SP        random                   DT    0.802    0.497   0.497  0.497  0.497  -0.006  0.497    0.019     0.001
  1  SP        ts                       DT    0.973    0.927   0.934  0.927  0.93    0.86   0.927    0.003     0
  2  SP        pck_size                 DT    0.999    0.997   0.996  0.997  0.997   0.994  0.997    0.001     0
  3  SP        Ether_type               DT    0.89     0.5     0.445  0.5    0.471   0      0.5      0         0
  4  SP        LLC_dsap                 DT    0.89     0.5     0.445  0.5    0.471   0      0.5      0.001     0
  5  SP        LLC_ssap                 DT    0.89     0.5     0.445  0.5    0.471   0      0.5      0         0
  6  SP        LLC_ctrl                 DT    0.89     0.5     0.445  0.5    0.471  

100%|████████████████████████████████████████████████████████████████████████████████| 174/174 [01:38<00:00,  1.77it/s]


     Attack    Feature                  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T
---  --------  -----------------------  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------
  0  OS        random                   DT    0.888    0.498   0.499  0.498  0.498  -0.003  0.498    0.019     0.001
  1  OS        ts                       DT    0.982    0.927   0.91   0.927  0.918   0.836  0.927    0.009     0
  2  OS        pck_size                 DT    0.964    0.724   0.903  0.724  0.783   0.569  0.724    0         0
  3  OS        Ether_type               DT    0.943    0.5     0.471  0.5    0.485   0      0.5      0         0.001
  4  OS        LLC_dsap                 DT    0.943    0.5     0.471  0.5    0.485   0      0.5      0         0
  5  OS        LLC_ssap                 DT    0.943    0.5     0.471  0.5    0.485   0      0.5      0         0
  6  OS        LLC_ctrl                 DT    0.943    0.5     0.471  0.5    0.4

100%|████████████████████████████████████████████████████████████████████████████████| 174/174 [01:31<00:00,  1.90it/s]

     Attack    Feature                  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T
---  --------  -----------------------  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------
  0  MHDis     random                   DT    0.961    0.506   0.507  0.506  0.507   0.013  0.506    0.014     0.001
  1  MHDis     ts                       DT    0.981    0.76    0.761  0.76   0.759   0.517  0.76     0.007     0
  2  MHDis     pck_size                 DT    0.998    0.999   0.958  0.999  0.977   0.955  0.999    0         0
  3  MHDis     Ether_type               DT    0.999    0.999   0.97   0.999  0.984   0.968  0.999    0.001     0
  4  MHDis     LLC_dsap                 DT    0.979    0.5     0.49   0.5    0.495   0      0.5      0.001     0
  5  MHDis     LLC_ssap                 DT    0.979    0.5     0.49   0.5    0.495   0      0.5      0         0
  6  MHDis     LLC_ctrl                 DT    0.979    0.5     0.49   0.5    0.495  

In [14]:
file_list={"SYN":['SM\\DoS-SYN-6.csv','SM\\DoS-SYN-5.csv'],
"ACK":['SM\\MB-ACK-4.csv','SM\\MB-ACK-3.csv'],
"BF":['SM\\MB-BF-3.csv','SM\\MB-BF-5.csv'],
"HTTP":['SM\\MB-HTTP-4.csv','SM\\MB-HTTP-2.csv'],
"UDP":['SM\\MB-UDP-3.csv','SM\\MB-UDP-2.csv'],
"ARP":['SM\\MitM-ARP-4.csv','SM\\MitM-ARP-3.csv'],
"SCHD":['SM\\Scan-HDis-3.csv','SM\\Scan-HDis-5.csv'],
"SP":['SM\\Scan-Port-3.csv','SM\\Scan-Port-4.csv'],
"OS":['SM\\Scan-OS-3.csv','SM\\Scan-OS-2.csv'],
"MHDis":['SM\\MB-HDis-3.csv','SM\\MB-HDis-5.csv']}

In [ ]:
for train in (file_list):
    lines=[["Attack","Feature","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
    for dname in tqdm(features):
        try:
            target_names=["attack","benign"]
            feature=[dname, 'Label']
            
            output_csv=f"./results/compare/SS/DT_{train}_{dname}.csv"
            temp=ML(file_list[train][0],file_list[train][1],output_csv,feature,dname,train)  
            temp=temp[2:]
            temp=[train,dname,"DT"]+temp
            lines.append(temp)
        except:
            print("#"*110)
            print(f"ERROR ABOUT {list[train]} Dataset - Feature {dname}")
            print("#"*110+"\n\n")

    results = pd.DataFrame (lines[1:], columns = lines[0])
    print (tabulate(results, headers=list(results.columns)))




100%|████████████████████████████████████████████████████████████████████████████████| 174/174 [12:32<00:00,  4.32s/it]


     Attack    Feature                  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
---  --------  -----------------------  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
  0  SYN       random                   DT    0.998    0.998   0.998  0.998  0.998  0.996  0.998    0.079     0.004
  1  SYN       ts                       DT    0.557    0.577   0.751  0.577  0.477  0.148  0.577    0.001     0
  2  SYN       pck_size                 DT    0.998    0.998   0.998  0.998  0.998  0.996  0.998    0.001     0.001
  3  SYN       Ether_type               DT    0.524    0.5     0.262  0.5    0.344  0      0.5      0.002     0
  4  SYN       LLC_dsap                 DT    0.524    0.5     0.262  0.5    0.344  0      0.5      0.001     0
  5  SYN       LLC_ssap                 DT    0.524    0.5     0.262  0.5    0.344  0      0.5      0         0
  6  SYN       LLC_ctrl                 DT    0.524    0.5     0.262  0.5    0.344  0   

 82%|█████████████████████████████████████████████████████████████████▋              | 143/174 [12:50<02:46,  5.38s/it]

In [ ]:
['C:/Users/kahra/Desktop/SON/003-mazebolt/SW/ACK_v3_SW.csv',
'C:/Users/kahra/Desktop/SON/003-mazebolt/SW/http-flood_SW.csv',
'C:/Users/kahra/Desktop/SON/003-mazebolt/SW/SYN_SW.csv',
'C:/Users/kahra/Desktop/SON/003-mazebolt/SW/udp_flood_SW.csv']

In [25]:
file_list={"SYN":['SM\\DoS-SYN-6.csv','C:/Users/kahra/Desktop/SON/003-mazebolt/SW/SYN_SW.csv'],
"ACK":['SM\\MB-ACK-4.csv','C:/Users/kahra/Desktop/SON/003-mazebolt/SW/ACK_v3_SW.csv'],
"BF":['SM\\MB-BF-3.csv',' '],
"HTTP":['SM\\MB-HTTP-4.csv','C:/Users/kahra/Desktop/SON/003-mazebolt/SW/http-flood_SW.csv'],
"UDP":['SM\\MB-UDP-3.csv','C:/Users/kahra/Desktop/SON/003-mazebolt/SW/udp_flood_SW.csv'],
"ARP":['SM\\MitM-ARP-4.csv',' '],
"SCHD":['SM\\Scan-HDis-3.csv',' '],
"SP":['SM\\Scan-Port-3.csv',' '],
"OS":['SM\\Scan-OS-3.csv',' '],
"MHDis":['SM\\MB-HDis-3.csv',' ']}

    Dataset      SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T
--  ---------  ----  ---  ----  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------
 0  random        0    1     0  DT    0.492    0.492   0.492  0.492  0.492  -0.015  0.492    0.09      0
 1  random        0    2     0  DT    0.503    0.502   0.502  0.502  0.502   0.004  0.502    0.131     0.002
 2  random        0    3     0  DT    0.488    0.488   0.488  0.488  0.488  -0.023  0.488    0.164     0.002
 3  random        0    4     0  DT    0.503    0.502   0.502  0.502  0.502   0.005  0.502    0.119     0.016
 4  random        0    5     0  DT    0.513    0.512   0.512  0.512  0.512   0.024  0.512    0.197     0
 5  random        0    1     1  DT    0.505    0.505   0.505  0.505  0.505   0.011  0.505    0.111     0.002
 6  random        0    2     1  DT    0.499    0.498   0.498  0.498  0.498  -0.004  0.498    0.074     0
 7  random        0    3     1 

    Dataset       SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  ----------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  Ether_type     0    1     0  DT    0.512      0.5   0.256    0.5  0.338      0    0.5    0.001     0
 1  Ether_type     0    2     0  DT    0.517      0.5   0.259    0.5  0.341      0    0.5    0.001     0
 2  Ether_type     0    3     0  DT    0.515      0.5   0.258    0.5  0.34       0    0.5    0.002     0
 3  Ether_type     0    4     0  DT    0.503      0.5   0.252    0.5  0.335      0    0.5    0         0.002
 4  Ether_type     0    5     0  DT    0.513      0.5   0.256    0.5  0.339      0    0.5    0.001     0
 5  Ether_type     0    1     1  DT    0.504      0.5   0.252    0.5  0.335      0    0.5    0         0
 6  Ether_type     0    2     1  DT    0.512      0.5   0.256    0.5  0.339      0    0.5    0         0
 7  Ether_type     0    3     1  DT    0.52

    Dataset      SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  ---------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  LLC_ctrl      0    1     0  DT    0.519      0.5   0.259    0.5  0.342      0    0.5    0.002     0
 1  LLC_ctrl      0    2     0  DT    0.512      0.5   0.256    0.5  0.339      0    0.5    0         0
 2  LLC_ctrl      0    3     0  DT    0.513      0.5   0.257    0.5  0.339      0    0.5    0         0
 3  LLC_ctrl      0    4     0  DT    0.511      0.5   0.256    0.5  0.338      0    0.5    0         0
 4  LLC_ctrl      0    5     0  DT    0.505      0.5   0.252    0.5  0.335      0    0.5    0.006     0
 5  LLC_ctrl      0    1     1  DT    0.519      0.5   0.26     0.5  0.342      0    0.5    0.003     0.001
 6  LLC_ctrl      0    2     1  DT    0.512      0.5   0.256    0.5  0.339      0    0.5    0.001     0
 7  LLC_ctrl      0    3     1  DT    0.51       0.5

    Dataset      SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  ---------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  EAPOL_len     0    1     0  DT    0.515      0.5   0.258    0.5  0.34       0    0.5    0         0
 1  EAPOL_len     0    2     0  DT    0.507      0.5   0.253    0.5  0.336      0    0.5    0.007     0.001
 2  EAPOL_len     0    3     0  DT    0.519      0.5   0.259    0.5  0.342      0    0.5    0.001     0
 3  EAPOL_len     0    4     0  DT    0.527      0.5   0.263    0.5  0.345      0    0.5    0.002     0
 4  EAPOL_len     0    5     0  DT    0.492      0.5   0.246    0.5  0.33       0    0.5    0         0
 5  EAPOL_len     0    1     1  DT    0.519      0.5   0.259    0.5  0.342      0    0.5    0.016     0
 6  EAPOL_len     0    2     1  DT    0.501      0.5   0.251    0.5  0.334      0    0.5    0.016     0
 7  EAPOL_len     0    3     1  DT    0.521      0.5

    Dataset      SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  ---------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  IP_tos        0    1     0  DT    0.505      0.5   0.253    0.5  0.336      0    0.5    0.002     0.001
 1  IP_tos        0    2     0  DT    0.506      0.5   0.253    0.5  0.336      0    0.5    0         0
 2  IP_tos        0    3     0  DT    0.524      0.5   0.262    0.5  0.344      0    0.5    0.01      0
 3  IP_tos        0    4     0  DT    0.51       0.5   0.255    0.5  0.338      0    0.5    0         0
 4  IP_tos        0    5     0  DT    0.515      0.5   0.257    0.5  0.34       0    0.5    0.002     0
 5  IP_tos        0    1     1  DT    0.507      0.5   0.254    0.5  0.337      0    0.5    0.001     0
 6  IP_tos        0    2     1  DT    0.528      0.5   0.264    0.5  0.345      0    0.5    0         0
 7  IP_tos        0    3     1  DT    0.521      0.5

...

    Dataset        SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  -----------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  TCP_ECE_sum     0    1     0  DT    0.516      0.5   0.258    0.5  0.34       0    0.5    0         0
 1  TCP_ECE_sum     0    2     0  DT    0.524      0.5   0.262    0.5  0.344      0    0.5    0.001     0.001
 2  TCP_ECE_sum     0    3     0  DT    0.495      0.5   0.247    0.5  0.331      0    0.5    0         0
 3  TCP_ECE_sum     0    4     0  DT    0.518      0.5   0.259    0.5  0.341      0    0.5    0.002     0
 4  TCP_ECE_sum     0    5     0  DT    0.507      0.5   0.254    0.5  0.337      0    0.5    0         0
 5  TCP_ECE_sum     0    1     1  DT    0.521      0.5   0.261    0.5  0.343      0    0.5    0         0
 6  TCP_ECE_sum     0    2     1  DT    0.504      0.5   0.252    0.5  0.335      0    0.5    0.001     0
 7  TCP_ECE_sum     0    3     1  

    Dataset          SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  -------------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  TCP_SYN_ratio     0    1     0  DT    0.995    0.995   0.995  0.995  0.995  0.99   0.995    0         0
 1  TCP_SYN_ratio     0    2     0  DT    0.999    0.999   0.999  0.999  0.999  0.998  0.999    0.016     0
 2  TCP_SYN_ratio     0    3     0  DT    0.997    0.997   0.997  0.997  0.997  0.994  0.997    0.003     0
 3  TCP_SYN_ratio     0    4     0  DT    0.997    0.997   0.997  0.997  0.997  0.994  0.997    0         0
 4  TCP_SYN_ratio     0    5     0  DT    0.997    0.997   0.997  0.997  0.997  0.995  0.997    0.003     0
 5  TCP_SYN_ratio     0    1     1  DT    0.995    0.995   0.995  0.995  0.995  0.99   0.995    0.003     0
 6  TCP_SYN_ratio     0    2     1  DT    0.998    0.998   0.998  0.998  0.998  0.995  0.998    0.005     0
 7  TCP_SYN_ratio   

    Dataset          SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  -------------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  TCP_ACK_ratio     0    1     0  DT    0.993    0.993   0.992  0.993  0.993  0.985  0.993    0.016     0
 1  TCP_ACK_ratio     0    2     0  DT    0.987    0.987   0.988  0.987  0.987  0.974  0.987    0.001     0
 2  TCP_ACK_ratio     0    3     0  DT    0.988    0.988   0.988  0.988  0.988  0.976  0.988    0.003     0.001
 3  TCP_ACK_ratio     0    4     0  DT    0.989    0.989   0.989  0.989  0.989  0.978  0.989    0.002     0
 4  TCP_ACK_ratio     0    5     0  DT    0.989    0.99    0.989  0.99   0.989  0.978  0.99     0         0
 5  TCP_ACK_ratio     0    1     1  DT    0.988    0.988   0.988  0.988  0.988  0.976  0.988    0         0
 6  TCP_ACK_ratio     0    2     1  DT    0.988    0.988   0.988  0.988  0.988  0.975  0.988    0.002     0
 7  TCP_ACK_rati

    Dataset          SW    T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T
--  -------------  ----  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------
 0  TCP_CWR_ratio     0    1     0  DT    0.509      0.5   0.254    0.5  0.337      0    0.5    0.001     0.001
 1  TCP_CWR_ratio     0    2     0  DT    0.507      0.5   0.254    0.5  0.337      0    0.5    0         0
 2  TCP_CWR_ratio     0    3     0  DT    0.509      0.5   0.255    0.5  0.337      0    0.5    0.001     0.001
 3  TCP_CWR_ratio     0    4     0  DT    0.513      0.5   0.257    0.5  0.339      0    0.5    0         0
 4  TCP_CWR_ratio     0    5     0  DT    0.521      0.5   0.261    0.5  0.343      0    0.5    0         0
 5  TCP_CWR_ratio     0    1     1  DT    0.515      0.5   0.257    0.5  0.34       0    0.5    0.002     0
 6  TCP_CWR_ratio     0    2     1  DT    0.513      0.5   0.257    0.5  0.339      0    0.5    0         0
 7  TCP_CWR_

KeyboardInterrupt: 

In [ ]:
for train in (file_list):
    lines=[["Attack","Feature","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
    for dname in tqdm(features):
        try:
            target_names=["attack","benign"]
            feature=[dname, 'Label']
            
            output_csv=f"./results/compare/DD/DT_{train}_{dname}.csv"
            temp=ML(file_list[train][0],file_list[train][1],output_csv,feature,dname,train)  
            temp=temp[2:]
            temp=[train,dname,"DT"]+temp
            lines.append(temp)
        except:
            print("#"*110)
            print(f"ERROR ABOUT {list[train]} Dataset - Feature {dname}")
            print("#"*110+"\n\n")

    results = pd.DataFrame (lines[1:], columns = lines[0])
    print (tabulate(results, headers=list(results.columns)))




In [ ]:
train = name_list[2]
test=  name_list[1]

iden,features=feature_names()

ml_list={"DT":DecisionTreeClassifier()}
step=1
flexible=0
i="0 empty"
output_csv="./isolated1/DT_"+i+".csv"
target_names=target_name(test)
ML_isolated(train,test,output_csv,features,step,flexible,i)   
for ii,i in enumerate(iden):
    temp=[i,"Label"]
    output_csv="./isolated1/DT_"+i+".csv"
    target_names=target_name(test)
    ML_isolated(train,test,output_csv,temp,step,flexible,i)   

In [ ]:
for ff in range(116):
    print(feature[0])
    for i in tqdm(range(2,21)):
        file_list=find_the_way("SW",f"last_{i}_")
        for ii in (file_list):
            temp=ii.split("_")[2]
            output_csv=f"./results/ML_{i}_{temp}.csv"
            target_names=["attack","benign"]
            dname=attack_dict[temp]  
            ML_CV(ii,"",output_csv,feature,dname,i)   
    
    name_list=find_the_way('./results/','.csv')
    lines=[["SW","Dataset","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
    cols=['Acc', 'b_Acc', 'Prec', 'Rec', 'F1','kap', 'ROC', 'tra-T', 'test-T']
    for p in name_list:
        df = pd.read_csv(p,usecols=cols) 
        temp=pd.DataFrame(df.mean())
        a=list(temp.values.flatten())
        add_data=p[:-4].split("_")[1:]
        add_data=[int(add_data[0]),add_data[1]]
        add_data=add_data+a
        lines.append(add_data)
    results = pd.DataFrame (lines[1:], columns = lines[0])
    results=results.sort_values('SW')
    #print (tabulate(results, headers=list(results.columns)))
    my_xticks=list(range(2,21))
    for j in ['F1']:#['Acc', 'b_Acc',  'F1', 'kap', 'ROC']:
        plt.figure(figsize=(10,5))
        graph_name=f"{ff}_all_Kitsune Dataset Attacks"
        plt.title(graph_name)
        plt.ylabel(evaluate[j])
        plt.xlabel("Sliding Window Size")
        for p in results["Dataset"].unique():
            df=results[results["Dataset"]==p]
            plt.plot(my_xticks,df[j], linestyle='-', marker='o',label= p)
        plt.legend(numpoints=1)
        plt.xticks(rotation=90) 
        graph_name=f"{ff}_all_Kitsune Dataset-{j} Attack.pdf"
        plt.savefig(graph_name,bbox_inches='tight',format="pdf")#, dpi=400)
        plt.show()
    my_xticks=list(range(2,21))
    for j in ['F1']:#['Acc', 'b_Acc',  'F1', 'kap', 'ROC']:
        fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
        for pp,p in enumerate(results["Dataset"].unique()):
            df=results[results["Dataset"]==p]
            axes[pp//3,pp%3].plot(my_xticks,df[j], linestyle='-', marker='o',label= p)
            axes[pp//3,pp%3].legend()
        fig.tight_layout()
        graph_name=f"{ff}_ind_Kitsune Dataset-{j} Attack.pdf"
        fig.savefig(graph_name,bbox_inches='tight',format="pdf")#, dpi=400)
        plt.show()
            
            
            
            
    feature.pop(0)

for ff in range(116):
    print(feature[0])
    for i in tqdm(range(2,21)):
        file_list=find_the_way("SW",f"last_{i}_")
        for ii in (file_list):
            temp=ii.split("_")[2]
            output_csv=f"./results/ML_{i}_{temp}.csv"
            target_names=["attack","benign"]
            dname=attack_dict[temp]  
            ML_CV(ii,"",output_csv,feature,dname,i)   
    
    name_list=find_the_way('./results/','.csv')
    lines=[["SW","Dataset","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
    cols=['Acc', 'b_Acc', 'Prec', 'Rec', 'F1','kap', 'ROC', 'tra-T', 'test-T']
    for p in name_list:
        df = pd.read_csv(p,usecols=cols) 
        temp=pd.DataFrame(df.mean())
        a=list(temp.values.flatten())
        add_data=p[:-4].split("_")[1:]
        add_data=[int(add_data[0]),add_data[1]]
        add_data=add_data+a
        lines.append(add_data)
    results = pd.DataFrame (lines[1:], columns = lines[0])
    results=results.sort_values('SW')
    #print (tabulate(results, headers=list(results.columns)))
    my_xticks=list(range(2,21))
    for j in ['F1']:#['Acc', 'b_Acc',  'F1', 'kap', 'ROC']:
        plt.figure(figsize=(10,5))
        graph_name=f"{ff}_all_Kitsune Dataset Attacks"
        plt.title(graph_name)
        plt.ylabel(evaluate[j])
        plt.xlabel("Sliding Window Size")
        for p in results["Dataset"].unique():
            df=results[results["Dataset"]==p]
            plt.plot(my_xticks,df[j], linestyle='-', marker='o',label= p)
        plt.legend(numpoints=1)
        plt.xticks(rotation=90) 
        graph_name=f"{ff}_all_Kitsune Dataset-{j} Attack.pdf"
        plt.savefig(graph_name,bbox_inches='tight',format="pdf")#, dpi=400)
        plt.show()
    my_xticks=list(range(2,21))
    for j in ['F1']:#['Acc', 'b_Acc',  'F1', 'kap', 'ROC']:
        fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 10))
        for pp,p in enumerate(results["Dataset"].unique()):
            df=results[results["Dataset"]==p]
            axes[pp//3,pp%3].plot(my_xticks,df[j], linestyle='-', marker='o',label= p)
            axes[pp//3,pp%3].legend()
        fig.tight_layout()
        graph_name=f"{ff}_ind_Kitsune Dataset-{j} Attack.pdf"
        fig.savefig(graph_name,bbox_inches='tight',format="pdf")#, dpi=400)
        plt.show()
            
            
            
            
    feature.pop(0)

In [76]:
features=[['ts', 'pck_size', 'payload_bytes','entropy'],
['dst_IP_diversity', 'dst_port_diversity', 'src_IP_diversity', 'IP_add_count'],
['sport', 'dport','sport_class', 'dport_class',
 'sport23', 'dport23', 'sport_bare', 'dport_bare','dport_sum', 'sport_sum'],
['TCP_FIN_ratio', 'TCP_SYN_ratio', 'TCP_RST_ratio', 'TCP_PSH_ratio', 
'TCP_ACK_ratio', 'TCP_URG_ratio', 'TCP_ECE_ratio', 'TCP_CWR_ratio'],
['TCP_FIN_SR', 'TCP_SYN_SR', 'TCP_RST_SR', 'TCP_PSH_SR', 'TCP_ACK_SR',
 'TCP_URG_SR', 'TCP_ECE_SR', 'TCP_CWR_SR'],
['pck_size_diff', 'ts_diff',    'TCP_window_diff','payload_bytes_diff' , 'entropy_diff'],
['pck_size_mean',  'ts_mean',  'TCP_window_mean','payload_bytes_mean','entropy_mean'],
['pck_size_std',   'ts_std', 'TCP_window_std','payload_bytes_std','entropy_std'],['pck_size_diff','pck_size_mean','pck_size_std','pck_size'],
['ts_diff', 'ts_mean', 'ts_std','ts'],
['TCP_window_diff','TCP_window_mean','TCP_window_std','TCP_window'],
 ['payload_bytes_diff' ,'payload_bytes_mean','payload_bytes_std','payload_bytes'],
    ['entropy_diff','entropy_mean','entropy_std','entropy'],
['pck_size_diff','pck_size_mean','pck_size_std',
'ts_diff', 'ts_mean', 'ts_std',
'TCP_window_diff','TCP_window_mean','TCP_window_std',
'payload_bytes_diff' ,'payload_bytes_mean','payload_bytes_std',
'entropy_diff','entropy_mean','entropy_std']]




In [77]:
attack_dict={'Active Wiretap':"AWT",
 'ARP MitM':"ARPMitM",
 'Fuzzing':"Fuzzing",
 'Mirai':"Mirai",
 'OS Scan':"OS_Scan",
 'SSDP Flood':"SSDP_F",
 'SSL Renegotiation':"SSL_Ren",
 'SYN DoS':"SYN_F",
 'Video Injection':"Vid_Inj"}

In [78]:
pp//3

0

for ff,feature in enumerate(features):
    feature.append("Label")
    for i in tqdm(range(2,21)):
        file_list=find_the_way("SW",f"last_{i}_")
        for ii in (file_list):
            temp=ii.split("_")[2]
            output_csv=f"./results/ML_{i}_{temp}.csv"
            target_names=["attack","benign"]
            dname=attack_dict[temp]  
            ML_CV(ii,"",output_csv,feature,dname,i)   

    name_list=find_the_way('./results/','.csv')
    lines=[["SW","Dataset","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T"]]
    cols=['Acc', 'b_Acc', 'Prec', 'Rec', 'F1','kap', 'ROC', 'tra-T', 'test-T']
    for p in name_list:
        df = pd.read_csv(p,usecols=cols) 
        temp=pd.DataFrame(df.mean())
        a=list(temp.values.flatten())
        add_data=p[:-4].split("_")[1:]
        add_data=[int(add_data[0]),add_data[1]]
        add_data=add_data+a
        lines.append(add_data)
    results = pd.DataFrame (lines[1:], columns = lines[0])
    results=results.sort_values('SW')
    
    #print (tabulate(results, headers=list(results.columns)))
    my_xticks=list(range(2,21))
    for j in ['F1']:#['Acc', 'b_Acc',  'F1', 'kap', 'ROC']:
        plt.figure(figsize=(10,5))
        graph_name=f"{ff}_all_Kitsune Dataset Attacks"
        plt.title(graph_name)
        plt.ylabel(evaluate[j])
        plt.xlabel("Sliding Window Size")
        for p in results["Dataset"].unique():

            df=results[results["Dataset"]==p]


            plt.plot(my_xticks,df[j], linestyle='-', marker='o',label= p)


        plt.legend(numpoints=1)
        plt.xticks(rotation=90) 
        graph_name=f"{ff}_all_Kitsune Dataset-{j} Attack.pdf"
        plt.savefig(graph_name,bbox_inches='tight',format="pdf")#, dpi=400)
        plt.show()
    my_xticks=list(range(2,21))
    for j in ['F1']:#['Acc', 'b_Acc',  'F1', 'kap', 'ROC']:
        fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))
        for pp,p in enumerate(results["Dataset"].unique()):
            df=results[results["Dataset"]==p]
            axes[pp%3].plot(my_xticks,df[j], linestyle='-', marker='o',label= p)
            axes[pp%3].legend()
        fig.tight_layout()
        graph_name=f"{ff}_ind_Kitsune Dataset-{j} Attack.pdf"
        fig.savefig(graph_name,bbox_inches='tight',format="pdf")#, dpi=400)
        plt.show()